In [1]:
#import os  
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import keras
from keras.datasets import cifar10
from keras import backend as K
from keras.layers import Input, Conv2D, Dense, BatchNormalization, Activation
from keras.layers import GlobalAveragePooling2D, MaxPooling2D, add
from keras.models import Model
from keras.layers import SeparableConv2D

from keras import optimizers,regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import he_normal
from keras.callbacks import LearningRateScheduler, TensorBoard, ModelCheckpoint

num_classes        = 10
batch_size         = 64         # 64 or 32 or other
epochs             = 300
iterations         = 782       
weight_decay=1e-4

log_filepath  = './xception_he_wd'

Using TensorFlow backend.


In [2]:
def color_preprocessing(x_train,x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    mean = [125.307, 122.95, 113.865]
    std  = [62.9932, 62.0887, 66.7048]
    for i in range(3):
        x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
        x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]
    return x_train, x_test

def scheduler(epoch):
    if epoch < 100:
        return 0.01
    if epoch < 200:
        return 0.001
    return 0.0001

# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test  = keras.utils.to_categorical(y_test, num_classes)
x_train, x_test = color_preprocessing(x_train, x_test)

In [3]:
# 36 convolutional layers are structured into 14 modules
def entryflow(x,params,top=False):
    # modules 2-4,13
    # params is (3,)
    # top = true means module 2, don't use relu
    residual = Conv2D(params[0], (1, 1), strides=(2, 2),padding='same',
                     kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(weight_decay))(x)
    residual = BatchNormalization(momentum=0.9, epsilon=1e-5)(residual)
    if top:
        x = Activation('relu')(x)
    x = SeparableConv2D(params[1], (3, 3),padding='same',
                       depthwise_initializer="he_normal",
                       pointwise_initializer="he_normal",
                       depthwise_regularizer=regularizers.l2(weight_decay),
                       pointwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(params[2], (3, 3),padding='same',
                       depthwise_initializer="he_normal",
                       pointwise_initializer="he_normal",
                       depthwise_regularizer=regularizers.l2(weight_decay),
                       pointwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = MaxPooling2D((3, 3), strides=(2, 2),padding='same')(x)
    x = add([x, residual])
    return x
    
def middleflow(x,params):
    # modules 5-12, params is int
    residual = x
    x = Activation('relu')(x)
    x = SeparableConv2D(params, (3, 3),padding='same',
                       depthwise_initializer="he_normal",
                       pointwise_initializer="he_normal",
                       depthwise_regularizer=regularizers.l2(weight_decay),
                       pointwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(params, (3, 3),padding='same',
                       depthwise_initializer="he_normal",
                       pointwise_initializer="he_normal",
                       depthwise_regularizer=regularizers.l2(weight_decay),
                       pointwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(params, (3, 3),padding='same',
                       depthwise_initializer="he_normal",
                       pointwise_initializer="he_normal",
                       depthwise_regularizer=regularizers.l2(weight_decay),
                       pointwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = add([x, residual])
    return x

def exitflow(x,params):
    # modules 14 , params is (2,)
    x = SeparableConv2D(params[0], (3, 3),padding='same',
                       depthwise_initializer="he_normal",
                       pointwise_initializer="he_normal",
                       depthwise_regularizer=regularizers.l2(weight_decay),
                       pointwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(params[1], (3, 3),padding='same',
                       depthwise_initializer="he_normal",
                       pointwise_initializer="he_normal",
                       depthwise_regularizer=regularizers.l2(weight_decay),
                       pointwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)   
    return x

In [4]:
def xception(img_input,shallow=False, classes=10):
    # modules 1
    x = Conv2D(32,(3, 3),strides=(2, 2),padding='same',
              kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(weight_decay))(img_input)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3),strides=(1,1),padding='same',
              kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    # module 2
    x = entryflow(x,(128,128,128),top=True)
    # module 3-4
    x = entryflow(x,(256,256,256))
    x = entryflow(x,(728,728,728))
    # module 5-12
    for _ in range(8):
        x = middleflow(x,728)
    # module 13
    x = entryflow(x,(1024,728,1024))
    # module 14
    x = exitflow(x,(1536,2048))
    # output
    x = GlobalAveragePooling2D()(x)
    x = Dense(classes, activation='softmax')(x)
    return x

In [5]:
img_input=Input(shape=(32,32,3))
output = xception(img_input)
model=Model(img_input,output)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 16, 16, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 16, 16, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 16, 16, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [6]:
# set optimizer
sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# set callback
tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=0)
change_lr = LearningRateScheduler(scheduler)
cbks = [change_lr,tb_cb]

# set data augmentation
datagen = ImageDataGenerator(horizontal_flip=True,
                             width_shift_range=0.125,
                             height_shift_range=0.125,
                             fill_mode='constant',cval=0.)
datagen.fit(x_train)

# start training
model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                    steps_per_epoch=iterations,
                    epochs=epochs,
                    callbacks=cbks,
                    validation_data=(x_test, y_test))
model.save('xception_he_wd.h5')

Epoch 1/300
782/782 [==============================] - 67s 86ms/step - loss: 6.1931 - acc: 0.3398 - val_loss: 5.7889 - val_acc: 0.4308
Epoch 2/300
782/782 [==============================] - 57s 73ms/step - loss: 6.1726 - acc: 0.3258 - val_loss: 6.1288 - val_acc: 0.3085
Epoch 3/300
782/782 [==============================] - 54s 70ms/step - loss: 6.1334 - acc: 0.3230 - val_loss: 5.6729 - val_acc: 0.3734
Epoch 4/300
782/782 [==============================] - 54s 70ms/step - loss: 5.8676 - acc: 0.3694 - val_loss: 5.6140 - val_acc: 0.4164
Epoch 5/300
782/782 [==============================] - 54s 69ms/step - loss: 5.6345 - acc: 0.4071 - val_loss: 5.4880 - val_acc: 0.4416
Epoch 6/300
782/782 [==============================] - 55s 71ms/step - loss: 5.5878 - acc: 0.3887 - val_loss: 5.3183 - val_acc: 0.4333
Epoch 7/300
782/782 [==============================] - 54s 69ms/step - loss: 5.5146 - acc: 0.3702 - val_loss: 5.4309 - val_acc: 0.3855
Epoch 8/300
782/782 [==============================] - 

782/782 [==============================] - 51s 65ms/step - loss: 0.7496 - acc: 0.9262 - val_loss: 2.0882 - val_acc: 0.6637
Epoch 122/300
782/782 [==============================] - 51s 65ms/step - loss: 0.7489 - acc: 0.9277 - val_loss: 2.0721 - val_acc: 0.6588
Epoch 123/300
782/782 [==============================] - 51s 65ms/step - loss: 0.7421 - acc: 0.9281 - val_loss: 2.0805 - val_acc: 0.6651
Epoch 124/300
782/782 [==============================] - 51s 65ms/step - loss: 0.7332 - acc: 0.9317 - val_loss: 2.1076 - val_acc: 0.6598
Epoch 125/300
782/782 [==============================] - 51s 65ms/step - loss: 0.7229 - acc: 0.9342 - val_loss: 2.0776 - val_acc: 0.6598
Epoch 126/300
782/782 [==============================] - 51s 65ms/step - loss: 0.7180 - acc: 0.9356 - val_loss: 2.1082 - val_acc: 0.6594
Epoch 127/300
782/782 [==============================] - 51s 65ms/step - loss: 0.7135 - acc: 0.9371 - val_loss: 2.1105 - val_acc: 0.6593
Epoch 128/300
782/782 [==============================] 

Epoch 181/300
782/782 [==============================] - 52s 66ms/step - loss: 0.5253 - acc: 0.9781 - val_loss: 2.2604 - val_acc: 0.6597
Epoch 182/300
782/782 [==============================] - 51s 66ms/step - loss: 0.5219 - acc: 0.9800 - val_loss: 2.2494 - val_acc: 0.6597
Epoch 183/300
782/782 [==============================] - 52s 66ms/step - loss: 0.5194 - acc: 0.9805 - val_loss: 2.3003 - val_acc: 0.6572
Epoch 184/300
782/782 [==============================] - 53s 68ms/step - loss: 0.5177 - acc: 0.9805 - val_loss: 2.2998 - val_acc: 0.6567
Epoch 185/300
782/782 [==============================] - 53s 68ms/step - loss: 0.5125 - acc: 0.9820 - val_loss: 2.2166 - val_acc: 0.6640
Epoch 186/300
782/782 [==============================] - 51s 66ms/step - loss: 0.5161 - acc: 0.9804 - val_loss: 2.2365 - val_acc: 0.6622
Epoch 187/300
782/782 [==============================] - 51s 66ms/step - loss: 0.5117 - acc: 0.9813 - val_loss: 2.2498 - val_acc: 0.6587
Epoch 188/300
782/782 [==================

Epoch 241/300
782/782 [==============================] - 52s 67ms/step - loss: 0.4534 - acc: 0.9942 - val_loss: 2.2602 - val_acc: 0.6588
Epoch 242/300
782/782 [==============================] - 52s 67ms/step - loss: 0.4541 - acc: 0.9938 - val_loss: 2.2359 - val_acc: 0.6636
Epoch 243/300
782/782 [==============================] - 53s 68ms/step - loss: 0.4531 - acc: 0.9942 - val_loss: 2.2306 - val_acc: 0.6609
Epoch 244/300
782/782 [==============================] - 52s 67ms/step - loss: 0.4533 - acc: 0.9940 - val_loss: 2.2343 - val_acc: 0.6641
Epoch 245/300
782/782 [==============================] - 51s 66ms/step - loss: 0.4530 - acc: 0.9942 - val_loss: 2.2359 - val_acc: 0.6623
Epoch 246/300
782/782 [==============================] - 50s 65ms/step - loss: 0.4523 - acc: 0.9942 - val_loss: 2.2424 - val_acc: 0.6602
Epoch 247/300
782/782 [==============================] - 51s 65ms/step - loss: 0.4520 - acc: 0.9943 - val_loss: 2.2075 - val_acc: 0.6630
Epoch 248/300
782/782 [==================